# Package Installation

In [1]:
!pip install -U peft datasets sacrebleu sentencepiece accelerate evaluate bitsandbytes transformers==4.44.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 114.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transfo

In [2]:
import pandas as pd
import sacrebleu
import numpy as np
import torch
import transformers
import peft
import datasets
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments


print("torch", torch.__version__)
print("transformers", transformers.__version__, "| peft", peft.__version__)
print("datasets", datasets.__version__)
print("GPU count:", torch.cuda.device_count())

2026-02-04 00:57:42.642388: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770166662.887522      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770166662.958386      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770166663.523459      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770166663.523501      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770166663.523504      24 computation_placer.cc:177] computation placer alr

torch 2.8.0+cu126
transformers 4.44.0 | peft 0.18.1
datasets 4.5.0
GPU count: 2


In [ ]:
from huggingface_hub import login
hf_token = ""
login(token=hf_token)

# Load data

In [4]:
from datasets import Dataset, DatasetDict
import pandas as pd

base_path = "/kaggle/input/vlsp-dataset/data"

def read_parallel(src_file, tgt_file):
    with open(base_path + src_file, encoding="utf-8") as f_src, open(base_path + tgt_file, encoding="utf-8") as f_tgt:
        src = f_src.read().strip().splitlines()
        tgt = f_tgt.read().strip().splitlines()
    n = min(len(src), len(tgt))
    return pd.DataFrame({'en': src[:n], 'vi': tgt[:n]})

train_df = read_parallel("/train.en.txt", "/train.vi.txt")
test_df  = read_parallel("/public_test.en.txt", "/public_test.vi.txt")
train_df = train_df.drop_duplicates().reset_index(drop=True)
test_df  = test_df.reset_index(drop=True)

raw = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "test": Dataset.from_pandas(test_df)
})
print(raw)

DatasetDict({
    train: Dataset({
        features: ['en', 'vi'],
        num_rows: 348224
    })
    test: Dataset({
        features: ['en', 'vi'],
        num_rows: 3000
    })
})


In [5]:
example_en = train_df.iloc[0]['en']
example_vi = train_df.iloc[0]['vi']
print(example_en, example_vi, sep='\n\n')

To evaluate clinical, subclinical symptoms of patients with otitis media with effusion and V.a at otorhinolaryngology department – Thai Nguyen national hospital

Nghiên cứu đặc điểm lâm sàng, cận lâm sàng bệnh nhân viêm tai ứ dịch trên viêm V.A tại Khoa Tai mũi họng - Bệnh viện Trung ương Thái Nguyên


# Prepare model

In [6]:
MODEL_NAME = "VietAI/envit5-translation"

In [7]:
from transformers import AutoModelForSeq2SeqLM

In [8]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True,
    trust_remote_code=True
)
model.config.use_cache = False

config.json:   0%|          | 0.00/721 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/accelerate/utils/modeling.py:1598: UserWarning: The following device_map keys do not match any submodules in the model: ['decoder.embed_tokens', 'encoder.embed_tokens']
  warnings.warn(


In [9]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# LoRA config

In [10]:
import re
def get_num_layers(model):
    numbers = set()
    for name, _ in model.named_parameters():
        for number in re.findall(r'\d+', name):
            numbers.add(int(number))
    return max(numbers)

def get_last_layer_linears(model):
    names = []
    
    num_layers = get_num_layers(model)
    for name, module in model.named_modules():
        if str(num_layers) in name and not "encoder" in name:
            if isinstance(module, torch.nn.Linear):
                names.append(name)
    return names

In [11]:
get_num_layers(model)

11

In [12]:
get_last_layer_linears(model)

['decoder.block.11.layer.0.SelfAttention.q',
 'decoder.block.11.layer.0.SelfAttention.k',
 'decoder.block.11.layer.0.SelfAttention.v',
 'decoder.block.11.layer.0.SelfAttention.o',
 'decoder.block.11.layer.1.EncDecAttention.q',
 'decoder.block.11.layer.1.EncDecAttention.k',
 'decoder.block.11.layer.1.EncDecAttention.v',
 'decoder.block.11.layer.1.EncDecAttention.o',
 'decoder.block.11.layer.2.DenseReluDense.wi_0',
 'decoder.block.11.layer.2.DenseReluDense.wi_1',
 'decoder.block.11.layer.2.DenseReluDense.wo']

In [13]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["k","q","v","o"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 3,538,944 || all params: 278,641,920 || trainable%: 1.2701


In [14]:
model

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(50048, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(50048, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=16, out_features=768, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
            

# Generate example

In [15]:
example_en = train_df.iloc[0]['en']
example_vi = train_df.iloc[0]['vi']
print(example_en, example_vi, sep='\n\n')

To evaluate clinical, subclinical symptoms of patients with otitis media with effusion and V.a at otorhinolaryngology department – Thai Nguyen national hospital

Nghiên cứu đặc điểm lâm sàng, cận lâm sàng bệnh nhân viêm tai ứ dịch trên viêm V.A tại Khoa Tai mũi họng - Bệnh viện Trung ương Thái Nguyên


In [16]:
def translate(text, model, tokenizer):
    prefix = "en: "
    input_text = prefix + text
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    
    with torch.no_grad():
        output_tokens = model.generate(
            **inputs, 
            max_length=512, 
            num_beams=5,
            early_stopping=True
        )
    
    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

print(f"en: {example_en}")
model.eval()
print(f"{translate(example_en, model, tokenizer)}")

en: To evaluate clinical, subclinical symptoms of patients with otitis media with effusion and V.a at otorhinolaryngology department – Thai Nguyen national hospital
vi: Đánh giá các triệu chứng lâm sàng, cận lâm sàng của bệnh nhân viêm tai giữa ứ dịch và viêm tai giữa ứ dịch tại Khoa Tai Mũi Họng-Bệnh viện Trung ương Thái Nguyên


# Format finetune data

In [17]:
def preprocess_function(examples):
    inputs = ["en: " + doc for doc in examples["en"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    targets = ["vi: " + doc for doc in examples["vi"]]
    labels = tokenizer(text_target=targets, max_length=512, truncation=True)

    labels_input_ids = labels["input_ids"]
    labels_input_ids = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label]
        for label in labels_input_ids
    ]
    
    model_inputs["labels"] = labels_input_ids
    return model_inputs

tokenized_raw = raw.map(
    preprocess_function,
    batched=True,
    remove_columns=raw["train"].column_names
)

Map:   0%|          | 0/348224 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [18]:
print(tokenized_raw["train"][0].keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [19]:
print(tokenized_raw["train"][0])

{'input_ids': [1055, 49804, 85, 18393, 14020, 49777, 3423, 43485, 884, 10594, 115, 7137, 299, 22973, 13506, 4502, 299, 40935, 9057, 111, 45, 49774, 49772, 413, 95, 21288, 1789, 223, 844, 5, 2326, 6710, 389, 5448, 25966, 4742, 8687, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [1875, 49804, 4148, 2850, 1945, 1410, 6103, 7690, 49777, 7081, 6103, 7690, 1719, 1027, 5604, 4419, 399, 1295, 802, 5604, 45, 49774, 49793, 672, 2365, 4418, 5672, 15628, 12, 1720, 2789, 1161, 8832, 2826, 2107, 1]}


# Train

In [20]:
import evaluate
import numpy as np

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    pad_to_multiple_of=8
)
metric = evaluate.load("sacrebleu")

def postprocess(preds, labels):
    preds = [p.strip() for p in preds]
    preds = [p.replace("vi:", "").replace("vi :", "").strip() for p in preds]
    
    labels = [l.strip() for l in labels]
    labels = [[l.replace("vi:", "").replace("vi :", "").strip()] for l in labels]
    
    preds = [' '.join(p.split()) for p in preds]
    labels = [[' '.join(l[0].split())] for l in labels]
    
    return preds, labels

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    preds = np.clip(preds, 0, len(tokenizer) - 1)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_preds, decoded_labels = postprocess(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [21]:
targs = Seq2SeqTrainingArguments(
    output_dir="./finetuned_model",
    fp16=True,       
    bf16=False,
    per_device_train_batch_size=8, 
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4, 
    learning_rate=2e-4,
    num_train_epochs=1,    
    logging_steps=250,
    save_strategy="steps",
    eval_strategy="epoch",
    save_steps=250,        
    save_total_limit=2,
    report_to="none",
    group_by_length=True, 
    ddp_find_unused_parameters=False,
    dataloader_num_workers=4,
    remove_unused_columns=False,
    predict_with_generate=True,
    generation_max_length=512
)

In [22]:
trainer = Seq2SeqTrainer(
    model=model,
    args=targs,
    train_dataset=tokenized_raw["train"],
    eval_dataset=tokenized_raw["test"], 
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [23]:
print("Start training...")
trainer.train()

Start training...


Epoch,Training Loss,Validation Loss,Bleu
1,1.019500,0.935738,47.082988


TrainOutput(global_step=10882, training_loss=1.0564780926752608, metrics={'train_runtime': 12842.8961, 'train_samples_per_second': 27.114, 'train_steps_per_second': 0.847, 'total_flos': 2.1256534494019584e+16, 'train_loss': 1.0564780926752608, 'epoch': 1.0})

In [24]:
for name, param in model.named_parameters():
    if "lora_A" in name:
        print(name, param.data.abs().mean().item())
        break

base_model.model.encoder.block.0.layer.0.SelfAttention.q.lora_A.default.weight 0.020981764420866966


In [25]:
def translate(text, model, tokenizer):
    prefix = "en: "
    input_text = prefix + text
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    
    with torch.no_grad():
        output_tokens = model.generate(
            **inputs, 
            max_length=512, 
            num_beams=5,
            early_stopping=True
        )
    
    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

print(f"en: {example_en}")
model.eval()
print(f"{translate(example_en, model, tokenizer)}")

en: To evaluate clinical, subclinical symptoms of patients with otitis media with effusion and V.a at otorhinolaryngology department – Thai Nguyen national hospital
vi: Đánh giá các triệu chứng lâm sàng, cận lâm sàng của bệnh nhân viêm tai giữa ứ dịch và viêm tai giữa ứ dịch tại khoa Tai Mũi Họng - Bệnh viện Trung ương Thái Nguyên


# Save model

In [26]:
trainer.save_model("/kaggle/working/final_translation_model")
tokenizer.save_pretrained("/kaggle/working/final_translation_model")

('/kaggle/working/final_translation_model/tokenizer_config.json',
 '/kaggle/working/final_translation_model/special_tokens_map.json',
 '/kaggle/working/final_translation_model/spiece.model',
 '/kaggle/working/final_translation_model/added_tokens.json')

# Loading and using the model later

In [27]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [28]:
path = "/kaggle/working/final_translation_model"
MODEL_NAME = "VietAI/envit5-translation"

In [29]:
tokenizer = AutoTokenizer.from_pretrained(path)
base_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME, device_map="auto")

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/usr/local/lib/python3.12/dist-packages/accelerate/utils/modeling.py:1598: UserWarning: The following device_map keys do not match any submodules in the model: ['decoder.embed_tokens', 'encoder.embed_tokens']
  warnings.warn(


In [30]:
model = PeftModel.from_pretrained(base_model, path)

In [31]:
def translate(text, model, tokenizer):
    prefix = "en: "
    input_text = prefix + text
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    
    with torch.no_grad():
        output_tokens = model.generate(
            **inputs, 
            max_length=512, 
            num_beams=5,
            early_stopping=True
        )
    
    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

print(f"en: {example_en}")
model.eval()
print(f"{translate(example_en, model, tokenizer)}")

en: To evaluate clinical, subclinical symptoms of patients with otitis media with effusion and V.a at otorhinolaryngology department – Thai Nguyen national hospital
vi: Đánh giá các triệu chứng lâm sàng, cận lâm sàng của bệnh nhân viêm tai giữa ứ dịch và viêm tai giữa ứ dịch tại khoa Tai Mũi Họng - Bệnh viện Trung ương Thái Nguyên
